In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


#  **import data & load library**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score,train_test_split, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay ,accuracy_score , classification_report

In [ ]:
data = pd.read_csv("/kaggle/input/titanic/train.csv")
print(data.shape)
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
print(test.shape)
test.head(5)

In [ ]:
test.describe()

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

# Missing Values # 

In [ ]:
data.isnull().sum()

In [ ]:
data.drop(columns = ["Cabin","Ticket","Name"],inplace =True)

In [ ]:
data = pd.get_dummies(data)

In [ ]:
imputer = SimpleImputer(strategy='median')

# Fit the imputer to the data
imputer.fit(data)

# Transform the data by imputing missing values
imputed_data = imputer.transform(data)

# Convert the NumPy array back to a DataFrame
data = pd.DataFrame(imputed_data, columns=data.columns)

In [ ]:
data.isnull().sum()

In [ ]:
data = pd.get_dummies(data)

In [ ]:
imputer = SimpleImputer(strategy='median')

# Fit the imputer to the data
imputer.fit(data)

# Transform the data by imputing missing values
imputed_data = imputer.transform(data)

# Convert the NumPy array back to a DataFrame
data = pd.DataFrame(imputed_data, columns=data.columns)

In [ ]:
data.head()

In [ ]:
data.info()

# Correlation # 

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(10, 8))
sns.heatmap(data.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax);

# Create & Train model # 

In [ ]:
X = data.drop('Survived', axis=1)
y= data['Survived']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=42)

In [ ]:
over_sampler = RandomOverSampler(random_state=42)
X_train_over, y_train_over = over_sampler.fit_resample(X_train, y_train)
print("X_train_over shape:", X_train_over.shape)
X_train_over.head()

In [ ]:
# Create and train the GradientBoostingClassifier model
model = GradientBoostingClassifier()


In [ ]:
cv_scores = cross_val_score(model, X_train_over, y_train_over, cv = 5, n_jobs=-1)
print(cv_scores)

In [ ]:
model.fit(X_train_over,y_train_over)
# Make predictions on the training set
y_train_pred = model.predict(X_train)

# Make predictions on the test set
y_test_pred = model.predict(X_test)
# Calculate the training and testing scores
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)

print("Training Score:", train_score)
print("Testing Score:", test_score)

In [ ]:
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test)

In [ ]:
class_report = classification_report(y_test, model.predict(X_test))
print(class_report)

In [ ]:
test.drop(columns = ["Cabin","Ticket","Name"],inplace =True)
test = pd.get_dummies(test)

In [ ]:
imputer = SimpleImputer(strategy='median')

# Fit the imputer to the data
imputer.fit(test)

# Transform the data by imputing missing values
imputed_data = imputer.transform(test)

# Convert the NumPy array back to a DataFrame
test = pd.DataFrame(imputed_data, columns=test.columns)

In [ ]:
test_ids = test["PassengerId"]

In [ ]:
submission_preds = model.predict(test)

In [ ]:
df = pd.DataFrame({
    "PassengerId": test_ids.values,
    "Survived" : submission_preds,
})

In [ ]:
df.to_csv("Submission.csv", index = False)